# Campo Lista Spark

Para usar pyspark en una notebook debemos tener

In [2]:
#pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark
findspark.init()

In [2]:
# Creamos la session de Spark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### Ejemplo

Les comparto un campo de ejemplo de una tabla con este tipo de estructura, como veran es un string plano

```sql
SELECT actividad_regimen_general
FROM xxxx
WHERE cuit_cuil = '20008688657'
```

In [16]:
#jsonString = 'Null'
df_json=spark.createDataFrame([(None, "2023-05-01")],["actividad_regimen_general","partition_date"])
df_json.show()

ValueError: Some of types cannot be determined after inferring

Bueno, ya tenemos la columna en un dataframe (este paso anterior no sera necesario porque ustedes se conectan al lago)

Ahora vamos a pasarle la estructura que deberia tener este texto plano, que es **como queremos que se comporte**

Al verlo intuimos lo siguiente: Una lista de estructuras, la cual se escribe en SPARK de la siguiente forma

In [14]:
schema = ArrayType(StructType(
      [
        StructField('descripcionActividad',  StringType(), True ),
        StructField('idActividad', StringType(), True),
        StructField('nomenclador', StringType(), True),
        StructField('orden', StringType(), True),
        StructField('periodo', StringType(), True)
      ]
    ))

#  creamos una nueva columna: withColumn("un alias", from_json("la columna a parsear", el_schema_creado))
mapped_df = df_json.withColumn("act", when(df_json.actividad_regimen_general == 'Null', 'no')
                               .otherwise(from_json("actividad_regimen_general", schema))   )   

AnalysisException: cannot resolve 'CASE WHEN (actividad_regimen_general = 'Null') THEN 'no' ELSE from_json(actividad_regimen_general) END' due to data type mismatch: THEN and ELSE expressions should all be same type or coercible to a common type, got CASE WHEN ... THEN string ELSE array<struct<descripcionActividad:string,idActividad:string,nomenclador:string,orden:string,periodo:string>> END;
'Project [actividad_regimen_general#0, partition_date#1, CASE WHEN (actividad_regimen_general#0 = Null) THEN no ELSE from_json(ArrayType(StructType(StructField(descripcionActividad,StringType,true),StructField(idActividad,StringType,true),StructField(nomenclador,StringType,true),StructField(orden,StringType,true),StructField(periodo,StringType,true)),true), actividad_regimen_general#0, Some(America/Buenos_Aires)) END AS act#85]
+- LogicalRDD [actividad_regimen_general#0, partition_date#1], false


In [5]:
mapped_df.show()

+-------------------------+--------------+----+
|actividad_regimen_general|partition_date| act|
+-------------------------+--------------+----+
|                     Null|    2023-05-01|null|
+-------------------------+--------------+----+



In [6]:
mapped_df.printSchema()

root
 |-- actividad_regimen_general: string (nullable = true)
 |-- partition_date: string (nullable = true)
 |-- act: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- descripcionActividad: string (nullable = true)
 |    |    |-- idActividad: string (nullable = true)
 |    |    |-- nomenclador: string (nullable = true)
 |    |    |-- orden: string (nullable = true)
 |    |    |-- periodo: string (nullable = true)



## Desarmar la Lista

Ahora con la funcion `explode` directamente al campo, creara un nuevo registro por cada elemento de la lista

In [13]:
mapped_df.select("act").show()

+----+
| act|
+----+
|null|
+----+



In [11]:
df2 = mapped_df.withColumn("act_exp", explode("act"))
df2.show()

+-------------------------+--------------+---+-------+
|actividad_regimen_general|partition_date|act|act_exp|
+-------------------------+--------------+---+-------+
+-------------------------+--------------+---+-------+



Para acceder a los campos dentro de las estructuras

Usamos la notacion punto `("x.y")`   

In [10]:
df2.select("act_exp.descripcionActividad","act_exp.orden","act_exp.periodo").show()

+--------------------+-----+--------+
|descripcionActividad|orden| periodo|
+--------------------+-----+--------+
|    CULTIVO DE TRIGO|  3.0|201311.0|
|     CULTIVO DE MAÍZ|  4.0|201311.0|
|CULTIVO DE PASTOS...|  6.0|201311.0|
|  CULTIVO DE GIRASOL|  5.0|201311.0|
|CRÍA DE GANADO BO...|  1.0|201311.0|
|INVERNADA  DE GAN...|  2.0|201311.0|
|SERVICIOS INMOBIL...|  7.0|201405.0|
+--------------------+-----+--------+



In [12]:
# Si quieren todo pueden usar el siempre u confiable (*)
df2.select("act_exp.*").show()

+--------------------+-----------+-----------+-----+--------+
|descripcionActividad|idActividad|nomenclador|orden| periodo|
+--------------------+-----------+-----------+-----+--------+
|    CULTIVO DE TRIGO|    11112.0|      883.0|  3.0|201311.0|
|     CULTIVO DE MAÍZ|    11121.0|      883.0|  4.0|201311.0|
|CULTIVO DE PASTOS...|    11130.0|      883.0|  6.0|201311.0|
|  CULTIVO DE GIRASOL|    11291.0|      883.0|  5.0|201311.0|
|CRÍA DE GANADO BO...|    14113.0|      883.0|  1.0|201311.0|
|INVERNADA  DE GAN...|    14114.0|      883.0|  2.0|201311.0|
|SERVICIOS INMOBIL...|   681098.0|      883.0|  7.0|201405.0|
+--------------------+-----------+-----------+-----+--------+

